In [2]:
import requests
import pandas as pd
import json

In [3]:
API_KEY = "67939bb0d275fb01694c238a5f6f7da5-3e9ae23411b17b20ead1a6ca001b5680"
ACCOUNT_ID = "101-002-27691953-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [13]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [18]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [21]:
response = session.get(url, params=None, data=None, headers=None)

In [22]:
response.status_code

200

In [23]:
data = response.json()

In [26]:
instruments_list = data['instruments']

In [27]:
len(instruments_list)

123

In [29]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [30]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 
         'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [33]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i }

In [35]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [36]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))